In [2]:
# Create Stash app client

import pandas as pd
import dotenv
import os
from libraries.client_stashapp import get_stashapp_client
from libraries.StashDbClient import StashDbClient

dotenv.load_dotenv()

stash = get_stashapp_client()
STASHAPP_URL = os.getenv("STASHAPP_URL")

stashbox_client = StashDbClient(
    os.getenv("STASHDB_ENDPOINT"),
    os.getenv("STASHDB_API_KEY"),
)

dUsing stash (v0.27.1-0) endpoint at http://localhost:6969/graphql
dPersisting Connection to Stash with ApiKey...


In [ ]:
# Utility functions
def create_modified_filter(data_quality_filter):
    depth = data_quality_filter['object_filter']['tags']['value']['depth']
    included_tag_ids = [tag['id'] for tag in data_quality_filter['object_filter']['tags']['value']['items']]
    excluded_tag_ids = [tag['id'] for tag in data_quality_filter['object_filter']['tags']['value']['excluded']]

    # Create the modified filter
    modified_filter = {
        'tags': {
            'modifier': 'INCLUDES_ALL',
            'value': included_tag_ids,
            'depth': depth,
            'excludes': excluded_tag_ids,
        }
    }
    return modified_filter

def get_scene_count(filter):
    result = stash.call_GQL("""query FindScenes($scene_filter: SceneFilterType) {
        findScenes(scene_filter: $scene_filter) {
            count
        }
    }""", variables={"scene_filter": filter})
    return result['findScenes']['count']

In [ ]:
# Get all saved filters
saved_filters = stash.call_GQL("""query FindSavedFilters {
    findSavedFilters {
        id
        mode
        name
        filter
        object_filter
        ui_options
    }
}""")

In [ ]:
# Get all data quality filters
data_quality_filters = [filter for filter in saved_filters['findSavedFilters'] if 'data quality' in filter['name'].lower()]

In [ ]:
data_quality_filters

In [ ]:
# Loop through all data quality filters and print out name and count of matching scenes
sorted_data_quality_filters = sorted(data_quality_filters, key=lambda x: x['name'])
for filter in sorted_data_quality_filters:
    name = filter['name']
    modified_filter = create_modified_filter(filter)
    scene_count = get_scene_count(modified_filter)
    print(f"{name}: {scene_count} scenes")


In [ ]:
twosome_lesbian_tags = stash.find_tag("Twosome (Lesbian)")
multiple_sex_scenes_tag = stash.find_tag("Multiple Sex Scenes In A Scene")
twosome_lesbian_scenes = stash.find_scenes({
    "tags": {"modifier": "INCLUDES_ALL", "value": [twosome_lesbian_tags['id']], "excludes": [multiple_sex_scenes_tag['id']]}
}, fragment="id title performers { id name gender } tags { id name }")




In [3]:
data_quality_issue_tag = stash.find_tag("Data Quality Issue")

In [11]:
all_scenes = stash.find_scenes({}, fragment="id title performers { id name gender } tags { id name }")
all_tags = stash.find_tags({}, fragment="id name description")

In [12]:
from abc import ABC, abstractmethod

class Rule(ABC):
    @abstractmethod
    def check(self, scene):
        pass
    
    def scene_has_any_tags(self, scene, tag_names):
        return any(tag['name'] in tag_names for tag in scene['tags'])

class GroupMakeUpRule(Rule):
    def __init__(self):
        self.tag_hierarchy = {
            "Solo": ["Solo Female", "Solo Male", "Solo Trans"],
            "Twosome": ["Twosome (Lesbian)", "Twosome (Straight)", "Twosome (Trans-Female)", "Twosome (Trans-Male)"],
            "Threesome": ["Threesome (Lesbian)", "Threesome (BBG)", "Threesome (BGG)"],
            "Foursome": ["Foursome (Lesbian)", "Foursome (BGGG)", "Foursome (BBGG)", "Foursome (BBBG)"],
        }
        self.special_rules = {
            "Solo Female": {
                "required_gender": ['FEMALE'],
                "exception_tags": ['Multiple Sex Scenes In A Scene', 'Non-Sex Performer']
            },
            "Solo Male": {
                "required_gender": ['MALE'],
                "exception_tags": ['Multiple Sex Scenes In A Scene', 'Non-Sex Performer']
            },
            "Solo Trans": {
                "required_gender": ['TRANS'],
                "exception_tags": ['Multiple Sex Scenes In A Scene', 'Non-Sex Performer']
            },
            "Twosome (Lesbian)": {
                "required_gender": ['FEMALE', 'FEMALE'],
                "exception_tags": ['Multiple Sex Scenes In A Scene', 'Non-Sex Performer']
            },
            "Twosome (Straight)": {
                "required_gender": ['FEMALE', 'MALE'],
                "exception_tags": ['Multiple Sex Scenes In A Scene', 'Non-Sex Performer', 'Missing Performer (Male)']
            },
            "Threesome (Lesbian)": {
                "required_gender": ['FEMALE', 'FEMALE', 'FEMALE'],
                "exception_tags": ['Multiple Sex Scenes In A Scene', 'Non-Sex Performer']
            },
            "Threesome (BBG)": {
                "required_gender": ['FEMALE', 'MALE', 'MALE'],
                "exception_tags": ['Multiple Sex Scenes In A Scene', 'Non-Sex Performer', 'Missing Performer (Male)']
            },
            "Threesome (BGG)": {
                "required_gender": ['FEMALE', 'FEMALE', 'MALE'],
                "exception_tags": ['Multiple Sex Scenes In A Scene', 'Non-Sex Performer', 'Missing Performer (Male)']
            },
            "Threesome (BBT)": {
                "required_gender": ['MALE', 'MALE', 'TRANSGENDER_FEMALE'],
                "exception_tags": ['Multiple Sex Scenes In A Scene', 'Non-Sex Performer']
            },
            "Threesome (BGT)": {
                "required_gender": ['MALE', 'FEMALE', 'TRANSGENDER_FEMALE'],
                "exception_tags": ['Multiple Sex Scenes In A Scene', 'Non-Sex Performer']
            },
            "Threesome (GGT)": {
                "required_gender": ['FEMALE', 'FEMALE', 'TRANSGENDER_FEMALE'],
                "exception_tags": ['Multiple Sex Scenes In A Scene', 'Non-Sex Performer']
            },
            "Threesome (GTT)": {
                "required_gender": ['FEMALE', 'TRANSGENDER_FEMALE', 'TRANSGENDER_FEMALE'],
                "exception_tags": ['Multiple Sex Scenes In A Scene', 'Non-Sex Performer']
            },
            "Threesome (Trans-Female)": {
                "required_gender": ['TRANSGENDER_FEMALE', 'TRANSGENDER_FEMALE', 'TRANSGENDER_FEMALE'],
                "exception_tags": ['Multiple Sex Scenes In A Scene', 'Non-Sex Performer']
            },
            "Foursome (BGGG)": {
                "required_gender": ['FEMALE', 'FEMALE', 'FEMALE', 'MALE'],
                "exception_tags": ['Multiple Sex Scenes In A Scene', 'Non-Sex Performer', 'Missing Performer (Male)']
            },
            "Foursome (BBGG)": {
                "required_gender": ['FEMALE', 'FEMALE', 'MALE', 'MALE'],
                "exception_tags": ['Multiple Sex Scenes In A Scene', 'Non-Sex Performer', 'Missing Performer (Male)']
            },
            "Foursome (BBBG)": {
                "required_gender": ['FEMALE', 'MALE', 'MALE', 'MALE'],
                "exception_tags": ['Multiple Sex Scenes In A Scene', 'Non-Sex Performer', 'Missing Performer (Male)']
            },
            "Foursome (Lesbian)": {
                "required_gender": ['FEMALE', 'FEMALE', 'FEMALE', 'FEMALE'],
                "exception_tags": ['Multiple Sex Scenes In A Scene', 'Non-Sex Performer']
            },
        }

    def check(self, scene):
        performer_genders = sorted([performer['gender'] for performer in scene['performers'] if performer['gender']])
        
        # Check tag hierarchy
        for parent_tag, child_tags in self.tag_hierarchy.items():
            if self.scene_has_any_tags(scene, child_tags) and not self.scene_has_any_tags(scene, [parent_tag]):
                return "Data Quality Issue: Group Makeup Specific But Not Generic"
            if self.scene_has_any_tags(scene, [parent_tag]) and not self.scene_has_any_tags(scene, child_tags):
                return "Data Quality Issue: Group Makeup Generic But Not Specific"

        # Check special rules
        for tag, rule in self.special_rules.items():
            if self.scene_has_any_tags(scene, [tag]):
                if 'required_gender' in rule and not self.scene_has_any_tags(scene, rule.get('exception_tags', [])):
                    if len(performer_genders) != len(rule['required_gender']):
                        return "Data Quality Issue: Incorrect Number of Performers Compared To Group Makeup"
                    elif performer_genders != rule['required_gender']:
                        return "Data Quality Issue: Incorrect Performer Genders Compared To Group Makeup"

        return None

class FacialRule(Rule):
    def check(self, scene):
        if (
            self.scene_has_any_tags(scene, ["Solo Female", "Twosome (Lesbian)", "Threesome (Lesbian)"]) and
            self.scene_has_any_tags(scene, ['Facial']) and
            not self.scene_has_any_tags(scene, ['Fake Cum', 'Jerk Off Instruction', 'Multiple Sex Scenes In A Scene'])
        ):
            return "Data Quality Issue: Incorrect Facial Tag"
        return None


rules = [
    FacialRule(),
    GroupMakeUpRule(),
    # Add more rules here as needed
]

def check_tag_rules(scenes, rules):
    all_violating_scenes = []
    for rule in rules:
        violating_scenes = []
        for scene in scenes:
            result = rule.check(scene)
            if result:
                violating_scenes.append({
                    'id': scene['id'],
                    'title': scene['title'],
                    'tags': [tag['name'] for tag in scene['tags']],  # Extract tag names
                    'tag_to_apply': result
                })
        all_violating_scenes.extend(violating_scenes)
    return all_violating_scenes

violating_scenes = check_tag_rules(all_scenes, rules)

print(f"Found {len(violating_scenes)} scenes violating the tag rules:")
for scene in violating_scenes:
    print(f"{scene['title']} {STASHAPP_URL}/scenes/{scene['id']}")
    print(f"Tags: {', '.join(scene['tags'])}")
    print(f"Tag to apply: {scene['tag_to_apply']}")
    print()
    
    refreshed_scene = stash.find_scene(scene['id'], fragment="performers { id name gender } tags { id name }")
    tags = [tag['id'] for tag in refreshed_scene['tags']]
    new_tag = stash.find_tag({"name": scene['tag_to_apply'], "parent_ids": [data_quality_issue_tag['id']]}, create=True)
    tags.append(new_tag['id'])
    stash.update_scene({
        "id": scene['id'],
        "tag_ids": tags
    })


iCould not find tag with name='Data Quality Issue: Incorrect Number of Performers Compared To Group Makeup' creating


Found 1470 scenes violating the tag rules:
 https://stash.chiefsclub.com/scenes/374
Tags: Missing: StashDB, Twosome, Twosome (Lesbian)
Tag to apply: Data Quality Issue: Incorrect Number of Performers Compared To Group Makeup

 https://stash.chiefsclub.com/scenes/375
Tags: Missing: StashDB, Twosome, Twosome (Lesbian)
Tag to apply: Data Quality Issue: Incorrect Number of Performers Compared To Group Makeup

 https://stash.chiefsclub.com/scenes/398
Tags: Missing: StashDB, Twosome, Twosome (Lesbian)
Tag to apply: Data Quality Issue: Incorrect Number of Performers Compared To Group Makeup

 https://stash.chiefsclub.com/scenes/445
Tags: Missing: StashDB, Twosome, Twosome (Lesbian)
Tag to apply: Data Quality Issue: Incorrect Number of Performers Compared To Group Makeup

 https://stash.chiefsclub.com/scenes/449
Tags: Missing: StashDB, Twosome, Twosome (Lesbian)
Tag to apply: Data Quality Issue: Incorrect Number of Performers Compared To Group Makeup

 https://stash.chiefsclub.com/scenes/472
T

iCould not find tag with name='Data Quality Issue: Group Makeup Generic But Not Specific' creating


 https://stash.chiefsclub.com/scenes/18282
Tags: AI_Tagged, Ass_AI, Belly Button_AI, Boobs_AI, Face_AI, Feet_AI, Fingers_AI, Grabbing Ass_AI, Grabbing Boobs_AI, Grabbing Hair/Head_AI, Lower Legs_AI, Masturbation_AI, Missing: StashDB, Pussy Rubbing_AI, Solo, Solo Female, Thighs_AI, Undressing_AI
Tag to apply: Data Quality Issue: Incorrect Number of Performers Compared To Group Makeup

 https://stash.chiefsclub.com/scenes/18292
Tags: Missing: StashDB, Threesome, Threesome (Lesbian)
Tag to apply: Data Quality Issue: Incorrect Number of Performers Compared To Group Makeup

 https://stash.chiefsclub.com/scenes/18302
Tags: Missing: StashDB, Threesome, Threesome (BBG)
Tag to apply: Data Quality Issue: Incorrect Number of Performers Compared To Group Makeup

 https://stash.chiefsclub.com/scenes/18303
Tags: Foursome, Foursome (Lesbian), Missing: StashDB
Tag to apply: Data Quality Issue: Incorrect Number of Performers Compared To Group Makeup

 https://stash.chiefsclub.com/scenes/18319
Tags: Mis

iCould not find tag with name='Data Quality Issue: Group Makeup Specific But Not Generic' creating


Maid For You Part 1 https://stash.chiefsclub.com/scenes/5834
Tags: 4K Available, Ass Grabbing, Bathroom, Bathtub, Belarusian, Breast Holding, Brown Hair, Chair, Couch, Creampie, Curly Hair, Czech, Dining Room, Drinking, Earrings, Flats, Foot Fetish, Indoors, Kissing, Kissing (FM), Kitchen, Leg Fetish, Living Room, Long Hair, Manicured, Markers: TPDB: Missing, Markers: timestamp.trade: Missing, Orgasm, Pussy Fingering, Pussy Licking, Series, Short Dress, Spread Legs, Striptease, Table, Towel, Trimmed Pussy, Twosome, Twosome (Straight), Upskirt, Wet Look
Tag to apply: Data Quality Issue: Incorrect Number of Performers Compared To Group Makeup

Sneak Peek: Production https://stash.chiefsclub.com/scenes/6039
Tags: 4K Available, Ass Grabbing, Bikini Panties, Breast Holding, Brown Hair, Camisole, Chair, Couch, Cowgirl, Cum on Stomach, Cumshot, Czech, Doggy Style, Drinking, Earrings, Erect Nipples, Foot Fetish, Handjob, Indoors, Interview, Jeans, Kissing, Kissing (FM), Leg Fetish, Living Room

iCould not find tag with name='Data Quality Issue: Incorrect Performer Genders Compared To Group Makeup' creating


The Ultimate Creampie - S4:E20 https://stash.chiefsclub.com/scenes/1227
Tags: 69 Creampie Eating, Black Hair, Blowjob, Creampie, Long Hair, Markers: TPDB: Missing, Markers: timestamp.trade: Saved, Twosome, Twosome (Straight)
Tag to apply: Data Quality Issue: Incorrect Performer Genders Compared To Group Makeup

Eager To Please - S11:E28 https://stash.chiefsclub.com/scenes/22586
Tags: Blowjob, High Heels, Markers: TPDB: Missing, Markers: timestamp.trade: Missing, Twosome, Twosome (Straight)
Tag to apply: Data Quality Issue: Incorrect Number of Performers Compared To Group Makeup

Pushing the limits Scene 1 https://stash.chiefsclub.com/scenes/1446
Tags: AI_TagMe, Markers: TPDB: Missing, Markers: timestamp.trade: Missing, Threesome, Threesome (BBG)
Tag to apply: Data Quality Issue: Incorrect Number of Performers Compared To Group Makeup

CHERIE DEVILLE - THE SWING LIFE https://stash.chiefsclub.com/scenes/6963
Tags: AI_TagMe, Markers: TPDB: Missing, Markers: timestamp.trade: Missing, Twoso

In [ ]:
# Check that all scenes have two performers who both have gender FEMALE
valid_scenes = []
invalid_scenes = []

for scene in twosome_lesbian_scenes:
    performers = scene['performers']
    if len(performers) == 2 and all(performer['gender'] == 'FEMALE' for performer in performers):
        valid_scenes.append(scene)
    else:
        invalid_scenes.append(scene)

print(f"Valid scenes: {len(valid_scenes)}")
print(f"Invalid scenes: {len(invalid_scenes)}")

if invalid_scenes:
    print("\nInvalid scenes:")
    for scene in invalid_scenes:
        print(f"ID: {scene['id']}")
        print(f"Title: {scene['title']}")
        print(f"Performers: {', '.join([f'{p['name']} ({p['gender']})' for p in scene['performers']])}")
        print()



In [ ]:
behind_the_scenes_tag = stash.find_tag("Behind the Scenes")
category_group_makeup_tag = stash.find_tag("Category: Group Makeup")
compilation_tag = stash.find_tag("Compilation")
full_movie_tag = stash.find_tag("Full Movie")
incoming_performers_tag = stash.find_tag("StashDB: Incoming performers")

scenes_without_group_makeup = stash.find_scenes({
    "tags": {
        "modifier": "INCLUDES_ALL",
        "value": [],
        "excludes": [
            behind_the_scenes_tag['id'],
            category_group_makeup_tag['id'],
            compilation_tag['id'],
            full_movie_tag['id']
        ]
    },
    "stash_id_endpoint": {"endpoint": "https://stashdb.org/graphql", "modifier": "NOT_NULL"},
}, fragment="id title performers { id name gender stash_ids { stash_id endpoint } } tags { id name } stash_ids { stash_id endpoint }")

for scene in scenes_without_group_makeup:
    scene_stashbox_id = next(
                (
                    sid["stash_id"]
                    for sid in scene["stash_ids"]
                    if sid.get("endpoint") == 'https://stashdb.org/graphql'
                ),
                None,
            )
    stashbox_scene = stashbox_client.query_scenes(scene_stashbox_id)
    
    scene_performer_ids = set([
        next(
            (
                sid["stash_id"]
                for sid in performer["stash_ids"]
                if sid.get("endpoint") == 'https://stashdb.org/graphql'
            ),
            None
        )
        for performer in scene['performers']
    ])
    
    stashbox_scene_performers = set([performer['performer']['id'] for performer in stashbox_scene['data']['findScene']['performers']])
    
    if not scene_performer_ids.issuperset(stashbox_scene_performers):
        print(scene['id'], scene['title'])
        print(scene['performers'])
        print(stashbox_scene_performers)
        
        refreshed_scene = stash.find_scene(scene['id'], fragment="performers { id name gender } tags { id name }")
        tags = [tag['id'] for tag in refreshed_scene['tags']]
        tags.append(incoming_performers_tag['id'])
        stash.update_scene({
            "id": scene['id'],
            "tag_ids": tags
        })
        
        print()
